In [7]:
import tensorflow as tf
import keras
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt 

data = []
labels = []


shared_dir = '/Users/prateek/Desktop/ML/Major/augmented/'
infected_dir = shared_dir + 'True_parasitized/'
uninfected_dir = shared_dir + 'True_uninfected/'
input_shape = [32, 32]
batch = 32

def reading(path):
    mode = cv2.IMREAD_GRAYSCALE
    img_read = cv2.imread(path)#, mode)
    # img_read = cv2.cvtColor(img_read, cv2.COLOR_BGR2RGB)
    img_read = np.float32(img_read)
    img_read = cv2.cvtColor(img_read, cv2.COLOR_BGR2HSV_FULL)
    img_resize = cv2.resize(img_read, input_shape)
    return img_resize

parasitized_data = os.listdir(infected_dir)
uninfected_data = os.listdir(uninfected_dir)

for img in parasitized_data:
    try:
        data.append(reading(infected_dir + img))
        labels.append(1)
    except Exception as e:
        print(e)
    
for img in uninfected_data:
    try:
        data.append(reading(uninfected_dir + img))
        labels.append(0)
    except Exception as e:
        print(e)

# shared_dir = '/Users/prateek/Desktop/ML/Major/augmented/'
# infected_dir = shared_dir + 'False_uninfected/'
# uninfected_dir = shared_dir + 'False_parasitized/'

# parasitized_data = os.listdir(infected_dir)
# uninfected_data = os.listdir(uninfected_dir)


# for img in parasitized_data:
#     try:
#         data.append(reading(infected_dir + img))
#         labels.append(1)
#     except Exception as e:
#         print(e)
    
# for img in uninfected_data:
#     try:
#         data.append(reading(uninfected_dir + img))
#         labels.append(0)
#     except Exception as e:
#         print(e)

image_data = np.array(data)
labels = np.array(labels)
# input_shape.append(3)

print("image_shape:", input_shape)
print("image_data:",len(image_data))
print("labels:",len(labels))

image_shape: [32, 32]
image_data: 261610
labels: 261610


In [8]:
bins = [i * 5 for i in range(0,36*2+1)]
print(bins)
histo = []
for image in image_data:
    hist, bin = np.histogram(image, bins=bins)
    histo.append(hist)
data = np.asarray(histo)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360]


In [9]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.1,random_state = 0)

print("X_train:",len(X_train))
print("X_test:",len(X_test))
print("y_train:",len(y_train))
print("y_test:",len(y_test))

X_train: 235449
X_test: 26161
y_train: 235449
y_test: 26161


In [10]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from keras import Model, Input
from keras.applications.xception import preprocess_input
from keras.callbacks import LearningRateScheduler
import numpy as np

epochs = 5
dropout_rate = 0.2
len_classes = len(bins)

model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='softmax'))


model.compile(optimizer='adam', loss='binary_crossentropy',metrics=keras.metrics.BinaryAccuracy())

In [11]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.1, verbose=10)

Epoch 1/5


2022-04-25 11:26:42.431668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 11:27:29.702751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [14]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
y_pred = model.predict(X_test)
print("loss", loss)
print("accuracy", accuracy)

818/818 [==============================] - 6s 8ms/step - loss: 0.3622 - binary_accuracy: 0.5041


2022-04-25 12:41:12.453865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


loss 0.36218100786209106
accuracy 0.5041474103927612


In [ ]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(confusion_matrix( y_test.argmax(axis=0), y_pred.argmax(axis=0) ))
print(accuracy_score(y_test, y_pred))
print( precision_score( y_test, y_pred, average='micro') )
print( recall_score( y_test, y_pred, average='micro') )